In [1]:
#importing neccessary modules
import pandas as pd 
import sqlite3
import numpy as np 
import requests 
from bs4 import BeautifulSoup as BS
import matplotlib.pyplot as plt
import tabulate

%matplotlib inline 
conn = sqlite3.connect('im.db')
cur = conn.cursor()

In [2]:
#seeing available files 
! ls

Phase 1 Project Notebook.ipynb
README.md
bom.movie_gross.csv.gz
im.db
rt.movie_info.tsv.gz
rt.reviews.tsv.gz
tmdb.movies.csv.gz
tn.movie_budgets.csv.gz


In [3]:
#opening and reviewing imdb datasets from movie_basics
imdb = pd.read_sql("""SELECT * FROM movie_basics;""", conn)
imdb.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [4]:
#dropping unnecessary columns from imdb movie_basics
imdb = imdb.drop('original_title', axis = 1)
imdb.head()

,movie_id,primary_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy"


In [5]:
#opening and reviewing imdb dataset from movie_akas
imdb2 = pd.read_sql("""SELECT * FROM movie_akas;""", conn)
imdb2.head()

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0


In [6]:
#dropping unnecessary columns from imdb movie_akas
imdb2 = imdb2.drop('ordering', axis = 1)
imdb2 = imdb2.drop('types', axis = 1)
imdb2 = imdb2.drop('attributes', axis = 1)
imdb2 = imdb2.drop('is_original_title', axis = 1)
imdb2.head()

,movie_id,title,region,language
0,tt0369610,Джурасик свят,BG,bg
1,tt0369610,Jurashikku warudo,JP,None
2,tt0369610,Jurassic World: O Mundo dos Dinossauros,BR,None
3,tt0369610,O Mundo dos Dinossauros,BR,None
4,tt0369610,Jurassic World,FR,None


In [7]:
#merging imdb datasets 
imdb_df = imdb.merge(imdb2, how='outer', on='movie_id')
imdb_df.head()

,movie_id,primary_title,start_year,runtime_minutes,genres,title,region,language
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",Sangharsh,IN,hi
1,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",Sunghursh,None,None
2,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",Sunghursh,IN,None
3,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",Sunghursh,IN,hi
4,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",Sungharsh,IN,hi


In [8]:
imdb_df.dtypes

movie_id            object
primary_title       object
start_year           int64
runtime_minutes    float64
genres              object
title               object
region              object
language            object
dtype: object

In [9]:
#opening and reviewing box office mojo dataset
gross = pd.read_csv("bom.movie_gross.csv.gz")
gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [10]:
#checking the columns datatypes and converting them to the appropriate type
gross['foreign_gross'] = pd.to_numeric(gross['foreign_gross'], errors = 'coerce')
gross.dtypes

title              object
studio             object
domestic_gross    float64
foreign_gross     float64
year                int64
dtype: object

In [11]:
#sorting values in descending order by foreign_gross
gross.sort_values(['foreign_gross'], ascending=False)

,title,studio,domestic_gross,foreign_gross,year
328,Harry Potter and the Deathly Hallows Part 2,WB,381000000.0,960500000.0,2011
1875,Avengers: Age of Ultron,BV,459000000.0,946400000.0,2015
727,Marvel's The Avengers,BV,623400000.0,895500000.0,2012
3081,Jurassic World: Fallen Kingdom,Uni.,417700000.0,891800000.0,2018
1127,Frozen,BV,400700000.0,875700000.0,2013
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [12]:
#Finding Q3 for foreign_gross
foreign_Q3 = gross['foreign_gross'].quantile(0.75)
print(foreign_Q3)

75050000.0


In [13]:
#limiting the gross dataset to show values only above the foreign_gross Q3
gross = gross[gross['foreign_gross'] > 75050000]
gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010
3,Inception,WB,292600000.0,535700000.0,2010
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010


In [14]:
#printing column names in each dataset to find which to merge on
print(imdb_df.columns.tolist())
print(gross.columns.tolist())

['movie_id', 'primary_title', 'start_year', 'runtime_minutes', 'genres', 'title', 'region', 'language']
['title', 'studio', 'domestic_gross', 'foreign_gross', 'year']


In [15]:
#merging the imdb and gross datasets on "title"
df = imdb_df.merge(gross, how='outer', on='title')
df.head()

,movie_id,primary_title,start_year,runtime_minutes,genres,title,region,language,studio,domestic_gross,foreign_gross,year
0,tt0063540,Sunghursh,2013.0,175.0,"Action,Crime,Drama",Sangharsh,IN,hi,NaN,NaN,NaN,NaN
1,tt3437682,Sangharsh,2014.0,NaN,Drama,Sangharsh,IN,None,NaN,NaN,NaN,NaN
2,tt0063540,Sunghursh,2013.0,175.0,"Action,Crime,Drama",Sunghursh,None,None,NaN,NaN,NaN,NaN
3,tt0063540,Sunghursh,2013.0,175.0,"Action,Crime,Drama",Sunghursh,IN,None,NaN,NaN,NaN,NaN
4,tt0063540,Sunghursh,2013.0,175.0,"Action,Crime,Drama",Sunghursh,IN,hi,NaN,NaN,NaN,NaN


In [16]:
#sorting the dataframe in descending order by foreign_gross
df.sort_values(['foreign_gross'], ascending=False)

,movie_id,primary_title,start_year,runtime_minutes,genres,title,region,language,studio,domestic_gross,foreign_gross,year
355552,NaN,NaN,NaN,NaN,NaN,Harry Potter and the Deathly Hallows Part 2,NaN,NaN,WB,381000000.0,960500000.0,2011.0
158192,tt2395427,Avengers: Age of Ultron,2015.0,141.0,"Action,Adventure,Sci-Fi",Avengers: Age of Ultron,DE,None,BV,459000000.0,946400000.0,2015.0
158194,tt2395427,Avengers: Age of Ultron,2015.0,141.0,"Action,Adventure,Sci-Fi",Avengers: Age of Ultron,IT,None,BV,459000000.0,946400000.0,2015.0
158195,tt2395427,Avengers: Age of Ultron,2015.0,141.0,"Action,Adventure,Sci-Fi",Avengers: Age of Ultron,CZ,None,BV,459000000.0,946400000.0,2015.0
158193,tt2395427,Avengers: Age of Ultron,2015.0,141.0,"Action,Adventure,Sci-Fi",Avengers: Age of Ultron,None,None,BV,459000000.0,946400000.0,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...
355540,tt9914254,A Cherry Tale,2019.0,85.0,Documentary,Kirsebæreventyret,DK,None,NaN,NaN,NaN,NaN
355541,tt9914254,A Cherry Tale,2019.0,85.0,Documentary,A Cherry Tale,XWW,en,NaN,NaN,NaN,NaN
355542,tt9915436,Vida em Movimento,2019.0,70.0,Documentary,Vida em Movimento,None,None,NaN,NaN,NaN,NaN
355543,tt9916170,The Rehearsal,2019.0,51.0,Drama,O Ensaio,BR,None,NaN,NaN,NaN,NaN


In [17]:
#limiting the dataset to movies whose foreign_gross was above Q3
df = df[df['foreign_gross'] > 75050000]
df.head()

,movie_id,primary_title,start_year,runtime_minutes,genres,title,region,language,studio,domestic_gross,foreign_gross,year
24122,tt0359950,The Secret Life of Walter Mitty,2013.0,114.0,"Adventure,Comedy,Drama",The Secret Life of Walter Mitty,US,None,Fox,58200000.0,129900000.0,2013.0
24123,tt0359950,The Secret Life of Walter Mitty,2013.0,114.0,"Adventure,Comedy,Drama",The Secret Life of Walter Mitty,None,None,Fox,58200000.0,129900000.0,2013.0
24241,tt0372538,Spy,2011.0,110.0,"Action,Crime,Drama",Spy,None,None,Fox,110800000.0,124800000.0,2015.0
24242,tt0372538,Spy,2011.0,110.0,"Action,Crime,Drama",Spy,US,None,Fox,110800000.0,124800000.0,2015.0
24243,tt3079380,Spy,2015.0,119.0,"Action,Comedy,Crime",Spy,GR,None,Fox,110800000.0,124800000.0,2015.0


In [18]:
#dropping unneccesary columns 
df = df.drop(['movie_id'], axis = 1)
df = df.drop(['primary_title'], axis =1)
df = df.drop(['start_year'], axis = 1)
df.head()

,runtime_minutes,genres,title,region,language,studio,domestic_gross,foreign_gross,year
24122,114.0,"Adventure,Comedy,Drama",The Secret Life of Walter Mitty,US,None,Fox,58200000.0,129900000.0,2013.0
24123,114.0,"Adventure,Comedy,Drama",The Secret Life of Walter Mitty,None,None,Fox,58200000.0,129900000.0,2013.0
24241,110.0,"Action,Crime,Drama",Spy,None,None,Fox,110800000.0,124800000.0,2015.0
24242,110.0,"Action,Crime,Drama",Spy,US,None,Fox,110800000.0,124800000.0,2015.0
24243,119.0,"Action,Comedy,Crime",Spy,GR,None,Fox,110800000.0,124800000.0,2015.0


In [19]:
#checking column types 
df.dtypes

runtime_minutes    float64
genres              object
title               object
region              object
language            object
studio              object
domestic_gross     float64
foreign_gross      float64
year               float64
dtype: object

In [20]:
#adding spaces between commas in genres
df['genres'] = df['genres'].astype('str')
df['genres'] = df['genres'].str.split(',') 
df['genres'] = df['genres'].dropna()
df.head()

,runtime_minutes,genres,title,region,language,studio,domestic_gross,foreign_gross,year
24122,114.0,"[Adventure, Comedy, Drama]",The Secret Life of Walter Mitty,US,None,Fox,58200000.0,129900000.0,2013.0
24123,114.0,"[Adventure, Comedy, Drama]",The Secret Life of Walter Mitty,None,None,Fox,58200000.0,129900000.0,2013.0
24241,110.0,"[Action, Crime, Drama]",Spy,None,None,Fox,110800000.0,124800000.0,2015.0
24242,110.0,"[Action, Crime, Drama]",Spy,US,None,Fox,110800000.0,124800000.0,2015.0
24243,119.0,"[Action, Comedy, Crime]",Spy,GR,None,Fox,110800000.0,124800000.0,2015.0


In [29]:
genre_count = df.groupby('genres')
genre_count.count()
print(genre_count)

TypeError: unhashable type: 'list'

In [21]:
#converting genres to tuple to make it hashable 
genres = tuple(list(df['genres']))
print(genres)

(['Adventure', 'Comedy', 'Drama'], ['Adventure', 'Comedy', 'Drama'], ['Action', 'Crime', 'Drama'], ['Action', 'Crime', 'Drama'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Adventure', 'Animation', 'Comedy'], ['Adventure', 'Animation', 'Comedy'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Thriller'], ['Action', 'Adventure', 

In [22]:
#getting unique values in genre

print ("The list of tuple is : " )
print(genres)

genre_unique = []
temp = set()
for genre in genres:
   for elem in genre:
      if not elem in temp:
         temp.add(elem)
         genre_unique.append(elem)
print("The unique elements in the list of tuples are : ")
print(genre_unique)

The list of tuple is : 
(['Adventure', 'Comedy', 'Drama'], ['Adventure', 'Comedy', 'Drama'], ['Action', 'Crime', 'Drama'], ['Action', 'Crime', 'Drama'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Action', 'Comedy', 'Crime'], ['Adventure', 'Animation', 'Comedy'], ['Adventure', 'Animation', 'Comedy'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Sci-Fi'], ['Action', 'Adventure', 'Thriller'], 

In [23]:
print(genre_unique)

['Adventure', 'Comedy', 'Drama', 'Action', 'Crime', 'Animation', 'Sci-Fi', 'Thriller', 'Family', 'Biography', 'History', 'Documentary', 'Fantasy', 'Sport', 'Horror', 'Mystery', 'Romance', 'Western', 'nan', 'Music', 'Musical', 'War']


In [30]:
print(genre)

['nan']
